In [ ]:
det_path = '/home/ubuntu/workspace/results/ab_car_validation_germany/faster_rcnn_cityscapes/detections.tsv'

In [ ]:
import glob
import pandas as pd
import pickle
import os

pickles_path = '/home/ubuntu/workspace/results/ab_car_validation_germany/riders/Co-DETR/all/backlog/Co-DETR/raw_detections/pickles'
output_path = '/home/ubuntu/workspace/results/ab_car_validation_germany/riders/Co-DETR/all/backlog/Co-DETR'

def parse_pickles_to_dataframe():
    data = []
    for path in glob.iglob(f"{pickles_path}/*.pickle"):
        with open(path, 'rb') as file:
            detections = pickle.load(file)
            for detection in detections:
                bbox_str = str(detection.bbox)
                coords_str = bbox_str[bbox_str.index('<')+1:bbox_str.index('>')].split(',')[0:4]
                x1, y1, x2, y2 = map(float, [coord.strip("()") for coord in coords_str])
                x_center = (x1 + x2) / 2
                y_center = (y1 + y2) / 2
                width = x2 - x1
                height = y2 - y1
                label = detection.category.id
                if label in [0, 1, 2, 3, 5, 7]:
                    if label in [1, 3]:
                        label = 1
                    elif label in [2, 5, 7]:
                        label = 2
                    score = detection.score.value*100
                    filename = os.path.basename(path)
                    data.append((filename.replace('.pickle', '.png'), x_center, y_center, width, height, label, score))
                    
    df = pd.DataFrame(data, columns=['name', 'x_center', 'y_center', 'width', 'height', 'label', 'score'])
    df[['is_occluded', 'is_truncated', 'd3_separation', 'is_rider_on_2_wheels']] = 0
    df[['l_label', 'r_label']] = 'None'
    df['threshold'] = 'None'
    df.to_csv(f"{output_path}/co_detr.tsv", sep='\t',index=False)
    print(f"{output_path}/co_detr.tsv")

In [ ]:
parse_pickles_to_dataframe()

In [ ]:
import pandas as pd

In [ ]:
detr = pd.read_csv(f"{output_path}/co_detr.tsv", sep='\t')
detr['label'].value_counts()

In [ ]:
det = pd.read_csv('/home/ubuntu/workspace/results/ab_car_validation_germany/docker/2023_07_26__20_32/b2b_det.tsv', sep='\t')
det['label'].value_counts()

In [ ]:
det = det[det['name'] == img_name]
det

In [ ]:
import pickle

path = '/home/ubuntu/workspace/results/ab_car_validation_germany/faster_rcnn_cityscapes/backlog/Co-DETR/raw_detections/pickles/ab_car_munich_urban_day_009_001710.pickle'
with open(path, 'rb') as file:
    detections = pickle.load(file)
detections

In [1]:
import pandas as pd
det_path ='/home/ubuntu/workspace/results/ab_car_validation_germany/faster_rcnn_cityscapes/try/detections.tsv'

In [6]:
det = pd.read_csv(det_path, sep='\t')
det.label.value_counts()

2    42
1     4
0     3
4     1
Name: label, dtype: int64

In [3]:
def remap_category(label, data_type):
    if data_type == "cityscapes":
        mapping = {
            'person': 0, 
            'car': 2, 
            'bicycle': 1,   
            'rider': 4,   
            'truck': 2,  
            'bus': 2,   
            'motorcycle': 1
            }
        new_label = mapping.get(label, -1)
    elif data_type == "coco":
        mapping = {
            'person': 0,  
            'bicycle': 1,  
            'car': 2,  
            'motorcycle': 1, 
            'bus': 2,  
            'truck': 2  
        }
        new_label = mapping.get(label, -1)
    return new_label

In [4]:
det_remapped = det.copy()
det_remapped['label'] = det['label'].apply(lambda label: remap_category(label, "cityscapes"))

det_remapped.head()

,name,x_center,y_center,width,height,label,score
0,1662977743942218.png,1753.894653,950.715485,39.239502,18.003601,-1,14.611559
1,1662979168727275.png,1820.570007,993.032715,96.047974,83.685791,-1,99.994230
2,1662979168727275.png,1898.974060,972.736877,70.590942,48.454102,-1,98.170072
3,1662979168727275.png,895.262604,957.108124,382.445984,114.521057,-1,84.483677
4,1662979168727275.png,1776.969116,971.231506,27.625488,33.808350,-1,83.978945


In [5]:
det_remapped.label.value_counts()

-1    50
Name: label, dtype: int64

In [46]:
import torchvision
import torch
import numpy as np
import pandas as pd

class NMSProcessor:
    def __init__(self, iou_thresh=0.5):
        self.iou_thresh = iou_thresh

    def convert_to_corners(self, boxes):
        """ Convert boxes from center format to corners format """
        x_center, y_center, width, height = boxes.T
        x1 = x_center - (width / 2)
        y1 = y_center - (height / 2)
        x2 = x_center + (width / 2)
        y2 = y_center + (height / 2)
        return np.stack((x1, y1, x2, y2), axis=1)

    def nms(self, boxes, scores, labels):
        boxes_corners = self.convert_to_corners(boxes)
        idxs = torchvision.ops.nms(torch.Tensor(boxes_corners), torch.Tensor(scores), iou_threshold=self.iou_thresh).numpy()
        return boxes[idxs], scores[idxs], labels[idxs]

    def nms_by_category(self, df):
        if df.empty:
            return df
        
        boxes = df[['x_center', 'y_center', 'width', 'height']].values
        scores = df['score'].values
        labels = df['label'].values
        
        boxes_list, scores_list, labels_list = [], [], []
        
        for clz in [0, 1, 2, 4]:
            mask = (labels == clz)
            
            if mask.sum() > 0:
                boxes_clz = boxes[mask]
                scores_clz = scores[mask]
                labels_clz = labels[mask]
                
                boxes_nms, scores_nms, labels_nms = self.nms(boxes_clz, scores_clz, labels_clz)
                
                boxes_list.extend(boxes_nms)
                scores_list.extend(scores_nms)
                labels_list.extend(labels_nms)

        result = pd.DataFrame({
            'name': df['name'].iloc[0],
            'x_center': [box[0] for box in boxes_list],
            'y_center': [box[1] for box in boxes_list],
            'width': [box[2] for box in boxes_list],
            'height': [box[3] for box in boxes_list],
            'score': scores_list,
            'label': labels_list
        })
        
        return result


In [47]:
len(det)

9

In [51]:
import pandas as pd
import cv2
import os


root = '/home/ubuntu/workspace/results/ab_car_validation_germany/faster_rcnn_cityscapes/infer'
det_path = f'{root}/detections.tsv'
base_img_path = '/home/ubuntu/workspace/datasets/ab_car_validation_germany/images'

det = pd.read_csv(det_path, sep='\t')
# det = det[det['name'] == '1662977544906087.png'].reset_index(drop=True)
nms_processor = NMSProcessor(iou_thresh=0.6)
det_nms = det.groupby('name').apply(nms_processor.nms_by_category).reset_index(drop=True)

unique_images = det['name'].unique()

for img_name in unique_images[:]:
    img_file_path = os.path.join(base_img_path, img_name)
    img = cv2.imread(img_file_path)
    
    if img is None:
        print(f"Failed to read {img_file_path}")
        continue
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_basename = os.path.basename(img_name).split('.')[0] 

    raw_output_path = f'{root}/{img_basename}_raw.png'
    cv2.imwrite(raw_output_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    print(raw_output_path)

    for df, tag in zip([det, det_nms], ['orig', 'nms']):
        img_copy = img.copy()
        image_detections = df[df['name'] == img_name]
        
        for _, detection in image_detections.iterrows():
            x_cent = int(detection['x_center'])
            y_cent = int(detection['y_center'])
            width = int(detection['width'])
            height = int(detection['height'])
            label = str(detection['label'])
            score = float(detection['score'])

            x1 = int(x_cent - width / 2)
            y1 = int(y_cent - height / 2)
            x2 = int(x_cent + width / 2)
            y2 = int(y_cent + height / 2)
            cv2.rectangle(img_copy, (x1, y1), (x2, y2), (0, 255, 0), 2)

            text = f"{label}: {score:.2f}"
            cv2.putText(img_copy, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        output_path = f'{root}/{img_basename}_det_{tag}.png'
        cv2.imwrite(output_path, cv2.cvtColor(img_copy, cv2.COLOR_RGB2BGR))
        print(output_path)


/home/ubuntu/workspace/results/ab_car_validation_germany/faster_rcnn_cityscapes/try/1662977743942218_raw.png
/home/ubuntu/workspace/results/ab_car_validation_germany/faster_rcnn_cityscapes/try/1662977743942218_det_orig.png
/home/ubuntu/workspace/results/ab_car_validation_germany/faster_rcnn_cityscapes/try/1662977743942218_det_nms.png
/home/ubuntu/workspace/results/ab_car_validation_germany/faster_rcnn_cityscapes/try/1662979168727275_raw.png
/home/ubuntu/workspace/results/ab_car_validation_germany/faster_rcnn_cityscapes/try/1662979168727275_det_orig.png
/home/ubuntu/workspace/results/ab_car_validation_germany/faster_rcnn_cityscapes/try/1662979168727275_det_nms.png
/home/ubuntu/workspace/results/ab_car_validation_germany/faster_rcnn_cityscapes/try/1662977022898784_raw.png
/home/ubuntu/workspace/results/ab_car_validation_germany/faster_rcnn_cityscapes/try/1662977022898784_det_orig.png
/home/ubuntu/workspace/results/ab_car_validation_germany/faster_rcnn_cityscapes/try/1662977022898784_det_

In [52]:
det

,name,x_center,y_center,width,height,label,score
0,1662977743942218.png,1753.894653,950.715485,39.239502,18.003601,2,14.611559
1,1662979168727275.png,1820.570007,993.032715,96.047974,83.685791,2,99.994230
2,1662979168727275.png,1898.974060,972.736877,70.590942,48.454102,2,98.170072
3,1662979168727275.png,895.262604,957.108124,382.445984,114.521057,2,84.483677
4,1662979168727275.png,1776.969116,971.231506,27.625488,33.808350,2,83.978945
5,1662979168727275.png,1747.121094,967.602051,43.709961,24.273560,2,66.614634
6,1662979168727275.png,1767.064758,969.722168,24.743286,27.276489,2,57.277191
7,1662979168727275.png,1788.000061,972.915070,41.933228,45.662659,2,39.812165
8,1662979168727275.png,812.171295,988.332886,53.264343,55.624268,1,27.477458
9,1662977022898784.png,1477.402649,990.514954,48.022095,15.334839,2,73.632342


In [53]:
det_nms

,name,x_center,y_center,width,height,score,label
0,1662977022898784.png,1477.402649,990.514954,48.022095,15.334839,73.632342,2
1,1662977022898784.png,1609.587891,982.635803,47.442627,29.693970,10.103514,2
2,1662977115900082.png,1226.041321,949.416168,312.113647,155.687195,99.985147,2
3,1662977115900082.png,1583.156433,981.920319,46.056030,14.138733,42.912456,2
4,1662977115900082.png,1333.642090,990.204559,88.009277,25.442078,31.813613,2
5,1662977482005204.png,1830.578979,998.954163,169.398682,154.129761,99.986386,2
6,1662977482005204.png,2160.086182,859.848999,409.249512,426.745605,95.660859,2
7,1662977482005204.png,1725.008728,941.825867,163.354126,41.598511,57.494879,2
8,1662977482005204.png,1937.406677,985.243408,57.440552,55.575562,28.112417,2
9,1662977482005204.png,1920.652893,983.289490,30.892700,53.654419,27.893847,2


In [72]:
det_path = '/home/ubuntu/workspace/results/ab_car_validation_germany/faster_rcnn_cityscapes/infer/detections.tsv'
det = pd.read_csv(det_path, sep = '\t')
det.label.value_counts()

2    36102
0    18553
1     6037
4     2281
Name: label, dtype: int64

In [73]:
det[det['label'] == 4] = 0
det[['d3_separation', 'r_label', 'l_label']] = 0
det.label.value_counts()

2    36102
0    20834
1     6037
Name: label, dtype: int64

In [74]:
det.to_csv(det_path.replace('.tsv', '_merged.tsv'), sep='\t', index=False)

In [75]:
det = pd.read_csv(det_path.replace('.tsv', '_merged.tsv'), sep = '\t')
det.label.value_counts()

2    36102
0    20834
1     6037
Name: label, dtype: int64

In [76]:
det_path.replace('.tsv', '_merged.tsv')

'/home/ubuntu/workspace/results/ab_car_validation_germany/faster_rcnn_cityscapes/infer/detections_merged.tsv'

In [77]:
det.head()

,name,x_center,y_center,width,height,score,label,d3_separation,r_label,l_label
0,1662976481024636.png,1865.803528,985.089752,250.966919,60.439880,96.843332,2,0,0,0
1,1662976481024636.png,2809.960449,974.153564,35.310547,30.455933,92.511737,2,0,0,0
2,1662976481024636.png,1928.747742,970.929504,136.258423,37.207520,56.500942,2,0,0,0
3,1662976481024636.png,1497.641785,991.174713,302.036987,80.560730,43.277952,2,0,0,0
4,1662976481024636.png,1680.262695,974.681671,266.356201,75.438782,20.816495,2,0,0,0


In [2]:
from sahi.predict import get_prediction, get_sliced_prediction, predict
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [3]:
predict(
    model_type='mmdet',
    model_path='/home/ubuntu/git/Co-DETR/weights/mask_rcnn_r50_fpn_1x_cityscapes_20201211_133733-d2858245.pth',
    model_config_path='/home/ubuntu/git/Co-DETR/configs/cityscapes/mask_rcnn_r50_fpn_1x_cityscapes.py',
    model_device=device,
    model_confidence_threshold=0.1,
    source='/home/ubuntu/workspace/datasets/ab_car_validation_germany/images_samp',
    slice_height=1280,
    slice_width=1280,
    overlap_height_ratio=0.1,
    overlap_width_ratio=0.1,
    novisual=True, 
    export_pickle=True,
    project='/home/ubuntu/workspace/results/ab_car_validation_germany/mask_rcnn_cityscapes/infer',
    name='mask_trials',
    no_sliced_prediction=True 
)


There are 10 listed files in folder: images_samp/


/opt/conda/envs/sahi/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
Performing inference on images:   0%|          | 0/10 [00:00<?, ?it/s]/home/ubuntu/git/Co-DETR/mmdet/datasets/utils.py:66: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  warnings.warn(
Performing inference on images:   0%|          | 0/10 [00:01<?, ?it/s]


IndexError: index 4 is out of bounds for axis 0 with size 0